#Modeling - ResNet50

## 병합한 pickle파일 가져오기

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
from google.colab import drive
drive.mount('/content/drive')

In [38]:
!cp /content/drive/MyDrive/KDT/offline/mini_project3-image/강아지품종예측\(미니프로젝트\)/데이터수집\(크롤링\)/*.pickle /content/drive/MyDrive/KDT/offline/mini_project3-image/github/

In [40]:
with open('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/image.pickle','rb') as f:
  data = pickle.load(f)
with open('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/label.pickle','rb') as f:
  label = pickle.load(f)

In [41]:
data

array([[[[230.00604 , 233.00604 , 238.00604 ],
         [228.31111 , 231.31111 , 236.31111 ],
         [228.15555 , 231.15555 , 236.15555 ],
         ...,
         [221.      , 229.      , 236.      ],
         [221.      , 229.      , 236.      ],
         [219.      , 229.      , 236.      ]],

        [[227.76666 , 230.76666 , 235.76666 ],
         [229.35445 , 232.35445 , 237.35445 ],
         [230.76666 , 233.76666 , 238.76666 ],
         ...,
         [221.      , 229.      , 236.      ],
         [221.      , 229.      , 236.      ],
         [219.      , 229.      , 236.      ]],

        [[228.77777 , 231.77777 , 236.77777 ],
         [228.38889 , 231.38889 , 236.38889 ],
         [229.      , 232.      , 237.      ],
         ...,
         [221.      , 229.      , 236.      ],
         [221.      , 229.      , 236.      ],
         [219.      , 229.      , 236.      ]],

        ...,

        [[ 32.835705,  32.835705,  32.835705],
         [ 39.853752,  39.853752,  39.853752]

In [42]:
label

array(['0', '0', '0', ..., '13', '13', '13'], dtype='<U2')

In [47]:
np.unique(label)

array(['0', '1', '10', '11', '12', '13', '2', '3', '4', '5', '6', '7',
       '8', '9'], dtype='<U2')

In [ ]:
# label -= 1
# label

In [48]:
data.shape,label.shape

((7500, 180, 180, 3), (7500,))

## train, test set 분리

In [49]:
train_data, test_data, train_label, test_label = train_test_split(data, label, test_size = 0.2, stratify=label, random_state=50)

In [50]:
train_data.shape,test_data.shape

((6000, 180, 180, 3), (1500, 180, 180, 3))

보통 대부분의 pretrained model(VGG, ResNet 등)의 경우, color image를 염두에 두고 학습을 진행하여 weight와 bias를 얻음.

즉, input image shape에 반드시 (224, 224, 3)과 같이 3 channel을 포함해야한다.

그래서 1 channel인 image를 pretrained model에 input으로 넣으려고하면 에러뜸

In [35]:
# #224*224는 image사이즈가 너무 커서 ram이 터지기 때문에 좀 더 작은 사이즈로 넣어줌
# #224*224 -> 180*180
# resize = lambda x: tf.image.resize(x, (180, 180))
# train_data = resize(train_data)
# test_data = resize(test_data)
# train_data.shape,test_data.shape

(TensorShape([9031, 180, 180, 1]), TensorShape([2258, 180, 180, 1]))

1차원 → 3차원 → resize 순으로 하면 data가 너무 커서 오류가 발생한다. 

repeat이전 1차원에서 resize를 하고 3차원 순으로 하면 오류 해결됨.

즉,1차원 → resize → 3차원 순으로 

In [ ]:
# train_data = tf.repeat(train_data, 3, axis = 3) 
# train_data.shape

In [ ]:
# test_data = tf.repeat(test_data, 3, axis=3)
# test_data.shape

## zero centering 과 Data int 화진행

In [51]:
def preprocess(image,mean):
    image -= mean
    return np.array(image)

In [52]:
mean = np.mean(train_data, axis=0)

In [53]:
#zero-centering
train = preprocess(train_data,mean)
test = preprocess(test_data,mean)

In [54]:
#data int
train = train.astype(np.int32)
test = test.astype(np.int32)

In [55]:
train_label = train_label.astype(np.int32)
test_label = test_label.astype(np.int32)

## pre-trained 된 ResNet50 model을 갖고 하위층 일부 재학습하는 transfer learning

### ver.1

딥러닝 모델정의(하위층+분류층 학습) Dropout(0.5) 2번 사용한 모델
- Conv layer 12층
- 분류층

In [97]:
back_model = ResNet50(include_top=False, input_shape = (180,180 ,3), weights = 'imagenet')
back_model.trainable = False

for layer in back_model.layers[-12:]:
  layer.trainable = True

inputs = tf.keras.Input(shape=(180,180,3))

x = back_model(inputs, training=False)

# FC층 학습
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(14,activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)
model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

### 딥러닝 모델 훈련

In [70]:
model.fit(train,train_label, epochs=10,batch_size=128,validation_data=(test,test_label))

Epoch 1/10
47/47 [==============================] - 25s 434ms/step - loss: 2.7522 - accuracy: 0.1720 - val_loss: 1.6826 - val_accuracy: 0.4633
Epoch 2/10
47/47 [==============================] - 17s 358ms/step - loss: 1.8023 - accuracy: 0.3848 - val_loss: 1.0483 - val_accuracy: 0.6847
Epoch 3/10
47/47 [==============================] - 17s 369ms/step - loss: 1.4202 - accuracy: 0.5138 - val_loss: 0.7171 - val_accuracy: 0.8027
Epoch 4/10
47/47 [==============================] - 17s 363ms/step - loss: 1.1438 - accuracy: 0.6050 - val_loss: 0.6225 - val_accuracy: 0.8287
Epoch 5/10
47/47 [==============================] - 17s 355ms/step - loss: 0.9809 - accuracy: 0.6643 - val_loss: 0.5583 - val_accuracy: 0.8333
Epoch 6/10
47/47 [==============================] - 17s 356ms/step - loss: 0.8400 - accuracy: 0.7003 - val_loss: 0.5293 - val_accuracy: 0.8473
Epoch 7/10
47/47 [==============================] - 17s 360ms/step - loss: 0.7916 - accuracy: 0.7205 - val_loss: 0.5104 - val_accuracy: 0.8460

### 모델 저장

In [71]:
model.save('/content/drive/MyDrive/KDT/offline/mini_project3-image/github//temp/final_model.h5')

In [72]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.4614 - accuracy: 0.8733 - 4s/epoch - 92ms/step

테스트 정확도: 0.8733333349227905


### fine-tuning

In [98]:
model = tf.keras.models.load_model('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/temp/final_model.h5')

In [99]:
## fine-tune
model.trainable = True # bottlenect feature 학습진행한다

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train, train_label, epochs = 10, validation_data=(test, test_label), batch_size= 128)

Epoch 1/10
47/47 [==============================] - 54s 1s/step - loss: 0.9380 - accuracy: 0.7165 - val_loss: 0.5250 - val_accuracy: 0.8460
Epoch 2/10
47/47 [==============================] - 47s 997ms/step - loss: 0.5966 - accuracy: 0.8065 - val_loss: 0.4519 - val_accuracy: 0.8633
Epoch 3/10
47/47 [==============================] - 47s 1s/step - loss: 0.4189 - accuracy: 0.8680 - val_loss: 0.4803 - val_accuracy: 0.8640
Epoch 4/10
47/47 [==============================] - 47s 1s/step - loss: 0.3621 - accuracy: 0.8883 - val_loss: 0.4460 - val_accuracy: 0.8793
Epoch 5/10
47/47 [==============================] - 47s 1s/step - loss: 0.2970 - accuracy: 0.9098 - val_loss: 0.4993 - val_accuracy: 0.8713
Epoch 6/10
47/47 [==============================] - 47s 1s/step - loss: 0.2494 - accuracy: 0.9265 - val_loss: 0.6316 - val_accuracy: 0.8587
Epoch 7/10
47/47 [==============================] - 47s 1s/step - loss: 0.1899 - accuracy: 0.9428 - val_loss: 0.6023 - val_accuracy: 0.8773
Epoch 8/10
47/47 

In [100]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.5528 - accuracy: 0.8767 - 4s/epoch - 91ms/step

테스트 정확도: 0.8766666650772095


### ver.2

딥러닝 모델정의(하위층+분류층 학습) Dropout(0.5) 1번 사용한 모델
- Conv layer 12층
- 분류층

In [88]:
back_model = ResNet50(include_top=False, input_shape = (180,180 ,3), weights = 'imagenet')
back_model.trainable = False

for layer in back_model.layers[-12:]:
  layer.trainable = True

inputs = tf.keras.Input(shape=(180,180,3))

x = back_model(inputs, training=False)

# FC층 학습
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(14,activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [75]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 6, 6, 2048)        23587712  
                                                                 
 flatten_7 (Flatten)         (None, 73728)             0         
                                                                 
 dense_18 (Dense)            (None, 128)               9437312   
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 14)                1806      
                                                                 
Total params: 33,026,830
Trainable params: 9,439,118
Non-tr

### 딥러닝 모델 훈련

In [76]:
model.fit(train,train_label, epochs=10,batch_size=128,validation_data=(test,test_label))

Epoch 1/10
47/47 [==============================] - 20s 369ms/step - loss: 1.6366 - accuracy: 0.5173 - val_loss: 0.5559 - val_accuracy: 0.8367
Epoch 2/10
47/47 [==============================] - 17s 361ms/step - loss: 0.6432 - accuracy: 0.7920 - val_loss: 0.4800 - val_accuracy: 0.8500
Epoch 3/10
47/47 [==============================] - 17s 371ms/step - loss: 0.4278 - accuracy: 0.8547 - val_loss: 0.4418 - val_accuracy: 0.8727
Epoch 4/10
47/47 [==============================] - 17s 358ms/step - loss: 0.3010 - accuracy: 0.8985 - val_loss: 0.4408 - val_accuracy: 0.8740
Epoch 5/10
47/47 [==============================] - 17s 354ms/step - loss: 0.2271 - accuracy: 0.9218 - val_loss: 0.4504 - val_accuracy: 0.8820
Epoch 6/10
47/47 [==============================] - 17s 358ms/step - loss: 0.1867 - accuracy: 0.9347 - val_loss: 0.4638 - val_accuracy: 0.8873
Epoch 7/10
47/47 [==============================] - 17s 363ms/step - loss: 0.1601 - accuracy: 0.9470 - val_loss: 0.4529 - val_accuracy: 0.8847

### 모델 저장

In [77]:
model.save('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/temp/final_model(V2).h5')

In [78]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.4775 - accuracy: 0.8907 - 4s/epoch - 91ms/step

테스트 정확도: 0.890666663646698


### fine-tuning

In [89]:
model = tf.keras.models.load_model('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/temp/final_model(V2).h5')

In [90]:
## fine-tune
model.trainable = True # bottlenect feature 학습진행한다

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train, train_label, epochs = 10, validation_data=(test, test_label), batch_size= 128)

Epoch 1/10
47/47 [==============================] - 54s 1s/step - loss: 0.4357 - accuracy: 0.8657 - val_loss: 0.4669 - val_accuracy: 0.8720
Epoch 2/10
47/47 [==============================] - 47s 1000ms/step - loss: 0.2173 - accuracy: 0.9287 - val_loss: 0.4590 - val_accuracy: 0.8820
Epoch 3/10
47/47 [==============================] - 48s 1s/step - loss: 0.1370 - accuracy: 0.9573 - val_loss: 0.5162 - val_accuracy: 0.8700
Epoch 4/10
47/47 [==============================] - 47s 1s/step - loss: 0.1390 - accuracy: 0.9555 - val_loss: 0.4740 - val_accuracy: 0.8713
Epoch 5/10
47/47 [==============================] - 47s 1s/step - loss: 0.0968 - accuracy: 0.9690 - val_loss: 0.4475 - val_accuracy: 0.8833
Epoch 6/10
47/47 [==============================] - 47s 1s/step - loss: 0.0935 - accuracy: 0.9723 - val_loss: 0.3933 - val_accuracy: 0.8893
Epoch 7/10
47/47 [==============================] - 47s 1s/step - loss: 0.0849 - accuracy: 0.9752 - val_loss: 0.4600 - val_accuracy: 0.8867
Epoch 8/10
47/47

In [91]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.4497 - accuracy: 0.8887 - 4s/epoch - 93ms/step

테스트 정확도: 0.8886666893959045


### ver.3

딥러닝 모델정의(하위층+분류층 학습) Dropout(0.5) 1번 사용한 모델
- Conv layer 30층
- 분류층


In [101]:
back_model = ResNet50(include_top=False, input_shape = (180,180 ,3), weights = 'imagenet')
back_model.trainable = False

for layer in back_model.layers[-30:]:
  layer.trainable = True

inputs = tf.keras.Input(shape=(180,180,3))

x = back_model(inputs, training=False)

# FC층 학습
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(14,activation='softmax')(x)

model = tf.keras.Model(inputs,outputs)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [93]:
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 6, 6, 2048)        23587712  
                                                                 
 flatten_11 (Flatten)        (None, 73728)             0         
                                                                 
 dense_26 (Dense)            (None, 256)               18874624  
                                                                 
 dropout_14 (Dropout)        (None, 256)               0         
                                                                 
 dense_27 (Dense)            (None, 14)                3598      
                                                                 
Total params: 42,465,934
Trainable params: 18,878,222
Non-

### 딥러닝 모델 훈련

In [81]:
model.fit(train,train_label, epochs=15,batch_size=128,validation_data=(test,test_label))

Epoch 1/15
47/47 [==============================] - 21s 380ms/step - loss: 1.3502 - accuracy: 0.6375 - val_loss: 0.4920 - val_accuracy: 0.8433
Epoch 2/15
47/47 [==============================] - 17s 371ms/step - loss: 0.4152 - accuracy: 0.8653 - val_loss: 0.4216 - val_accuracy: 0.8660
Epoch 3/15
47/47 [==============================] - 17s 371ms/step - loss: 0.2385 - accuracy: 0.9217 - val_loss: 0.4247 - val_accuracy: 0.8707
Epoch 4/15
47/47 [==============================] - 17s 357ms/step - loss: 0.1556 - accuracy: 0.9488 - val_loss: 0.4723 - val_accuracy: 0.8667
Epoch 5/15
47/47 [==============================] - 17s 357ms/step - loss: 0.1233 - accuracy: 0.9593 - val_loss: 0.4435 - val_accuracy: 0.8800
Epoch 6/15
47/47 [==============================] - 17s 363ms/step - loss: 0.0976 - accuracy: 0.9647 - val_loss: 0.4546 - val_accuracy: 0.8793
Epoch 7/15
47/47 [==============================] - 17s 365ms/step - loss: 0.0712 - accuracy: 0.9767 - val_loss: 0.5050 - val_accuracy: 0.8813

### 모델 저장

In [82]:
model.save('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/temp/final_model(V3).h5')

In [83]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.5610 - accuracy: 0.8827 - 4s/epoch - 92ms/step

테스트 정확도: 0.8826666474342346


### fine-tuning

In [102]:
model = tf.keras.models.load_model('/content/drive/MyDrive/KDT/offline/mini_project3-image/github/temp/final_model(V3).h5')

In [103]:
## fine-tune
model.trainable = True # bottlenect feature 학습진행한다

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train, train_label, epochs = 10, validation_data=(test, test_label), batch_size= 128)

Epoch 1/10
47/47 [==============================] - 54s 1s/step - loss: 0.2700 - accuracy: 0.9193 - val_loss: 0.5071 - val_accuracy: 0.8673
Epoch 2/10
47/47 [==============================] - 47s 1s/step - loss: 0.1217 - accuracy: 0.9627 - val_loss: 0.6203 - val_accuracy: 0.8413
Epoch 3/10
47/47 [==============================] - 48s 1s/step - loss: 0.1232 - accuracy: 0.9638 - val_loss: 0.4278 - val_accuracy: 0.8873
Epoch 4/10
47/47 [==============================] - 47s 1s/step - loss: 0.0761 - accuracy: 0.9728 - val_loss: 0.4827 - val_accuracy: 0.8793
Epoch 5/10
47/47 [==============================] - 47s 1s/step - loss: 0.0638 - accuracy: 0.9797 - val_loss: 0.4731 - val_accuracy: 0.8787
Epoch 6/10
47/47 [==============================] - 47s 1s/step - loss: 0.0727 - accuracy: 0.9780 - val_loss: 0.4458 - val_accuracy: 0.8753
Epoch 7/10
47/47 [==============================] - 47s 1s/step - loss: 0.0772 - accuracy: 0.9780 - val_loss: 0.4625 - val_accuracy: 0.8813
Epoch 8/10
47/47 [==

In [104]:
test_loss, test_acc = model.evaluate(test_data, test_label, verbose=2)
print('\n테스트 정확도:', test_acc)

47/47 - 4s - loss: 0.5280 - accuracy: 0.8880 - 4s/epoch - 92ms/step

테스트 정확도: 0.8880000114440918
